In [ ]:
import numpy as np
import pandas as pd
import re
from os import path
from datetime import datetime
from Bio import SeqIO
from Bio.Blast.Applications import NcbiblastnCommandline




---
## Build local BLAST database
#### only need to build once. Ignore this part if you have performed this before

In [ ]:
## change the name of genes that is going to be built into local database
gene_names = ['COI', 'COX3', 'CYTB', 'ND2', 'ND4']

for gene_name in gene_names:
    path_fastafile = 'GbRefgene/gb'+gene_name+'.fasta'
    dbtitle = gene_name+'ref'
    !makeblastdb -in {path_fastafile} -parse_seqids -title {dbtitle} -dbtype nucl



---
## Input Query filename and RefGene name
fasta file of sequencing reads

In [ ]:
from os import listdir
from os.path import isfile, join

mypath = 'SnowInput'  # change input files directory

fastq_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
fastq_files = [path.splitext(x)[0] for x in fastq_files]

print('Found %d sample files in directory: %s' % (len(fastq_files), mypath))


### Dereplication

In [ ]:
#fastq_files = ['Deer32']
#uncomment to manually input sample filenames without file extension

for eachfastq in fastq_files:
    !vsearch --derep_fulllength {mypath}/{eachfastq}.fa --output {mypath}/{eachfastq}_derep.fasta --sizeout --relabel uniq


### Frequency Filtering

In [ ]:
min_read_num = 1

for eachfastq in fastq_files:
    !vsearch --cluster_unoise {mypath}/{eachfastq}_derep.fasta --minsize {min_read_num} --unoise_alpha 2 --centroids {mypath}/{eachfastq}_freqfilt.fasta



In [ ]:
# query_files = ['Deer32']
# uncomment to manually change the filenames
query_files = fastq_files

gene_names = ['COI', 'COX3', 'CYTB', 'ND2', 'ND4']
# change the list of genes that is gonna be blasted upon


---
## Run Blast against Local Reference Databases

In [ ]:
percent_identity = 98   ## change the cutoff


for query_file in query_files:

    !mkdir {query_file}_blastResults

    path_blastin = mypath+'/'+query_file+'_freqfilt.fasta'
    
    for gene_name in gene_names:
        
        path_fastafile = 'GbRefgene/gb'+gene_name+'.fasta'
        path_blastout  = query_file+'_blastResults/'+query_file+'_blast'+gene_name+'.tsv'
        
        blastx_cline = NcbiblastnCommandline(query=path_blastin, db=path_fastafile, evalue=0.001, line_length=alignment_length, perc_identity=percent_identity, outfmt="6 qseqid qlen sseqid stitle pident length mismatch gapopen qstart qend sstart send evalue bitscore", out=path_blastout)
        print('\nBlasting '+query_file+' against', gene_name, 'database...', end=" ")
        t0 = datetime.now()
        stdout, stderr = blastx_cline()
        t1 = datetime.now()
        print('Completed. Runtime: ', t1 - t0)

    print('\n')



---
# Parse Blast results

In [ ]:
alignment_length = 50   ## cutoff of > 50 bp must be aligned


for query_file in query_files:
    print('Query File =', query_file)
    
    for gene_name in gene_names:
        blastn = pd.read_csv(query_file+'_blastResults/'+query_file+'_blast'+gene_name+'.tsv', sep='\t', header=None)
        blastn.columns = 'qseqid qlen sseqid stitle pident length mismatch gapopen qstart qend sstart send evalue bitscore'.split(' ')

        reads = set(blastn['qseqid'])
        print('Parsing blast ' + gene_name + ' result...')
        print('\t Number of reads with hits =', len(reads))


        neitheridx = []
        read_freq = []


        ## filter hits if it belongs to human, bacteria, or fungi
        for idx, eachrow in blastn.iterrows():
            if (idx+1) % 100000 == 0:    print(idx)

            m = re.search(';size=(\d+)', eachrow['qseqid'])
            read_count = m.group(1)
            read_freq.append(read_count)

            ## The presence of human, bacteria or fungi DNA could indicate contamination
            ## The interpretation varies according to the samples
            filtered_count = [0, 0, 0, 0]
            if 'Homo_sapiens' in eachrow['stitle']:
                filtered_count[0] += 1
            elif 'Bacteria' in eachrow['stitle']:
                filtered_count[1] += 1
            elif 'Fungi' in eachrow['stitle']:
                filtered_count[2] += 1
            elif eachrow['length'] < alignment_length:
                filtered_count[3] += 1
            else:
                neitheridx.append(idx)
                

        print("\t Hit_Count: Homo_sapiens = %d \t Bacteria = %d \t Fungi = %d \t Less than %dbp = %d" % (filtered_count[0], filtered_count[1], filtered_count[2], alignment_length, filtered_count[3]))



        ## retrieve frequency information
        blastn['read_freq'] = read_freq
        subset = blastn.loc[neitheridx, :]



        ## retrieve read sequence from query_file
        qseqids = list(set(list(subset['qseqid'])))
        qseqdict = dict()

        path_blastin = mypath+'/'+query_file+'_freqfilt.fasta'
        zipfa = SeqIO.parse(path_blastin, 'fasta')
        for read in zipfa:
            if read.id in qseqids:
                qseqdict[read.id] = read.seq

        qseqs = []
        for idx, eachrow in subset.iterrows():
            qseqs.append(str(qseqdict[eachrow['qseqid']]))
        subset['qseq'] = qseqs
        subset.to_csv(query_file+'_blastResults/'+query_file+'_blast'+gene_name+'_filtered.tsv', sep='\t')

    print('\n')


## Extract Top identity hit for each read

In [ ]:
for query_file in query_files:
    print('Query File =', query_file)
    print('Extract top hits of blast result')
    
    for gene_name in gene_names:
        print('\t Extracting gene', gene_name, '...')
        
        path_blastin = query_file+'_blastResults/'+query_file+'_blast'+gene_name+'_filtered.tsv'
        #if not path.exists(path_blastin):    continue
        ident = pd.read_csv(path_blastin, sep='\t', index_col=0)

        qseqids = []
        topidenIdx = []

        prev_qseqid = ''

        for idx, eachrow in ident.iterrows():
            qseqid = eachrow['qseqid']

            if qseqid != prev_qseqid:
                prev_qseqid = qseqid
                topident = eachrow['pident']
                topidenIdx.append(idx)

            else:
                if eachrow['pident'] == topident:
                    topidenIdx.append(idx)
        
        ident.loc[topidenIdx, :].to_csv(query_file+'_blastResults/'+query_file+'_blast'+gene_name+'-topIden.tsv', sep='\t')

    print('\t Completed\n')


## Counting Diet Species

In [ ]:
for query_file in query_files:
    print('Query File =', query_file)
    print('Counting species from top identity hits...')
    
    allprey = dict()
    for gidx, gene_name in enumerate(gene_names):
        print('\t Counting of gene', gene_name, '...')
        
        path_blastin = query_file+'_blastResults/'+query_file+'_blast'+gene_name+'-topIden.tsv'
        ident = pd.read_csv(path_blastin, sep='\t', index_col=0)

        for idx, eachrow in ident.iterrows():
            prey = eachrow['stitle'].split(';')[-1]
            read_freq = eachrow['read_freq']

            if prey not in allprey.keys():    allprey[prey] = [0]*len(gene_names)
            allprey[prey][gidx] += read_freq

    allprey = pd.DataFrame.from_dict(allprey, orient='index')
    allprey.columns = gene_names
    allprey.to_csv(query_file+'_blastResults/'+query_file+'_spcCount.tsv', sep='\t')

    print('\t Completed\n')
